<h1><p style="text-align: center;">Data Analysis with Python <br>Project - 1</p><h1> - Traffic Police Stops <br><img src="https://thumbs.dreamstime.com/z/traffic-police-officer-uk-vector-illustration-traffic-police-officer-uk-187594525.jpg" class="img-fluid" alt="CLRSWY" width="200" height="100"> 

Does the ``gender`` of a driver have an impact on police behavior during a traffic stop? **In this chapter**, we will explore that question while practicing filtering, grouping, method chaining, Boolean math, string methods, and more!

***

## Examining traffic violations

Before comparing the violations being committed by each gender, we should examine the ``violations`` committed by all drivers to get a baseline understanding of the data.

In this exercise, we'll count the unique values in the ``violation`` column, and then separately express those counts as proportions.

In [1]:
import pandas as pd
import numpy as np
import re

pol = pd.read_csv('police.csv.zip').sample(50000, random_state=101)
pol.shape

pol.drop(['state', 'county_name', 'county_fips', 'fine_grained_location'], axis=1, inplace=True)

pol.dropna(subset=['driver_gender'], inplace=True)

pol['is_arrested']=pol['is_arrested'].astype({'is_arrested':'bool'})

pol['stop_datetime'] = pol['stop_date'] + ' ' + pol['stop_time']

pol['stop_datetime'] = pd.to_datetime(pol['stop_datetime'])

pol.set_index('stop_datetime', inplace=True)

/Users/ozkanyildirim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
pol.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47180 entries, 2009-02-28 11:02:00 to 2013-03-21 11:54:00
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     47180 non-null  object 
 1   stop_date              47180 non-null  object 
 2   stop_time              47180 non-null  object 
 3   location_raw           47180 non-null  object 
 4   county_fips            0 non-null      float64
 5   fine_grained_location  0 non-null      float64
 6   police_department      47180 non-null  object 
 7   driver_gender          47180 non-null  object 
 8   driver_age_raw         47180 non-null  float64
 9   driver_age             47018 non-null  float64
 10  driver_race_raw        47180 non-null  object 
 11  driver_race            47180 non-null  object 
 12  violation_raw          47180 non-null  object 
 13  violation              47180 non-null  object 
 14  search_conducted   

In [3]:
pol['violation'].unique()

array(['Speeding', 'Registration/plates', 'Moving violation', 'Equipment',
       'Other', 'Seat belt'], dtype=object)

In [38]:
pol['violation'].value_counts(normalize=True, dropna=False)

Speeding               0.554769
Moving violation       0.189127
Equipment              0.129716
Other                  0.050890
Registration/plates    0.040971
Seat belt              0.034527
Name: violation, dtype: float64

***

## Comparing violations by gender

The question we're trying to answer is whether male and female drivers tend to commit different types of traffic violations.

In [5]:
polf = pol[pol['driver_gender'] == 'F']
polf.head()

,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2012-05-17 13:45:00,RI-2012-27184,2012-05-17,13:45,Zone K2,NaN,NaN,900,F,1991.0,21.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K2
2012-08-24 22:19:00,RI-2012-44496,2012-08-24,22:19,Zone X4,NaN,NaN,500,F,1990.0,22.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2006-11-20 13:45:00,RI-2006-51922,2006-11-20,13:45,Zone X3,NaN,NaN,200,F,1971.0,35.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X3
2013-10-21 11:33:00,RI-2013-36430,2013-10-21,11:33,Zone X3,NaN,NaN,200,F,1994.0,19.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X3
2008-07-21 10:59:00,RI-2008-27059,2008-07-21,10:59,Zone K2,NaN,NaN,900,F,1990.0,18.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K2


In [6]:
polm = pol[pol['driver_gender'] == 'M']
polm.head()

,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2009-02-28 11:02:00,RI-2009-08019,2009-02-28,11:02,Zone K2,NaN,NaN,900,M,1952.0,57.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K2
2008-08-30 18:09:00,RI-2008-32661,2008-08-30,18:09,Zone K3,NaN,NaN,300,M,1967.0,41.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K3
2009-04-13 22:36:00,RI-2009-14329,2009-04-13,22:36,Zone K1,NaN,NaN,600,M,1987.0,22.0,...,False,NaN,NaN,False,Citation,False,16-30 Min,True,False,Zone K1
2011-02-06 11:12:00,RI-2011-03623,2011-02-06,11:12,Zone X4,NaN,NaN,500,M,1988.0,23.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2011-01-22 07:43:00,RI-2011-02368,2011-01-22,07:43,Zone K3,NaN,NaN,300,M,1989.0,22.0,...,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3


In [7]:
polf['violation'].value_counts(normalize=True, dropna=False)

Speeding               0.650637
Moving violation       0.137819
Equipment              0.112492
Registration/plates    0.042107
Other                  0.028900
Seat belt              0.028045
Name: violation, dtype: float64

In [8]:
polm['violation'].value_counts(normalize=True, dropna=False)

Speeding               0.518800
Moving violation       0.208377
Equipment              0.136178
Other                  0.059141
Registration/plates    0.040544
Seat belt              0.036959
Name: violation, dtype: float64

***

## Comparing speeding outcomes by gender

When a driver is pulled over for speeding, many people believe that gender has an impact on whether the driver will receive a ticket or a warning. We will try to find evidences of this in the dataset.

First, we'll create two ``DataFrames`` of drivers who were stopped for ``speeding``: one containing ***females*** and the other containing ***males***.

Then, for each **gender**, we'll use the ``stop_outcome`` column to calculate what percentage of stops resulted in a ``"Citation"`` (meaning a ticket) versus a ``"Warning"``.

In [9]:
f_speed = polf[polf['violation'] == 'Speeding']
f_speed[['violation']].head()

,violation
stop_datetime,
2006-11-20 13:45:00,Speeding
2013-10-21 11:33:00,Speeding
2008-07-21 10:59:00,Speeding
2014-09-19 06:41:00,Speeding
2006-05-21 23:45:00,Speeding


In [10]:
m_speed = polm[polm['violation'] == 'Speeding']
m_speed[['violation']].head()

,violation
stop_datetime,
2009-02-28 11:02:00,Speeding
2009-04-13 22:36:00,Speeding
2011-01-22 07:43:00,Speeding
2005-12-20 04:00:00,Speeding
2008-08-26 14:12:00,Speeding


In [11]:
f_speed['stop_outcome'].value_counts()

Citation            7986
Warning              328
Arrest Driver         46
Arrest Passenger       7
N/D                    6
No Action              2
Name: stop_outcome, dtype: int64

In [12]:
f_speed['stop_outcome'].value_counts(normalize=True)

Citation            0.953552
Warning             0.039164
Arrest Driver       0.005493
Arrest Passenger    0.000836
N/D                 0.000716
No Action           0.000239
Name: stop_outcome, dtype: float64

In [13]:
m_speed['stop_outcome'].value_counts()

Citation            16807
Warning               626
Arrest Driver         305
Arrest Passenger       25
N/D                    19
No Action              17
Name: stop_outcome, dtype: int64

In [14]:
m_speed['stop_outcome'].value_counts(normalize=True)

Citation            0.944267
Warning             0.035171
Arrest Driver       0.017136
Arrest Passenger    0.001405
N/D                 0.001067
No Action           0.000955
Name: stop_outcome, dtype: float64

***

## Calculating the search rate

During a traffic stop, the police officer sometimes conducts a search of the vehicle. In this exercise, we'll calculate the percentage of all stops that result in a vehicle search, also known as the **search rate**.

In [15]:
pol['search_conducted'] = pol['search_conducted'].astype({'search_conducted':'bool'})
pol['search_conducted'].dtypes

dtype('bool')

In [16]:
pol['search_conducted'].value_counts()

False    45402
True      1778
Name: search_conducted, dtype: int64

In [17]:
pol['search_conducted'].value_counts(normalize=True)

False    0.962315
True     0.037685
Name: search_conducted, dtype: float64

In [18]:
pol['search_conducted'].mean()

0.03768545994065282

***

## Comparing search rates by gender

We'll compare the rates at which **female** and **male** drivers are searched during a traffic stop. 

First, we'll filter the ``DataFrame`` by gender and calculate the search rate for each group separately. Then, we'll perform the same calculation for both genders at once using a ``.groupby()``.

In [19]:
pol[pol['driver_gender']=='F']['search_conducted'].value_counts()

False    12625
True       247
Name: search_conducted, dtype: int64

In [20]:
pol[pol['driver_gender']=='F']['search_conducted'].value_counts(normalize=True)

False    0.980811
True     0.019189
Name: search_conducted, dtype: float64

In [21]:
pol[pol['driver_gender']=='M']['search_conducted'].value_counts()

False    32777
True      1531
Name: search_conducted, dtype: int64

In [22]:
pol[pol['driver_gender']=='M']['search_conducted'].value_counts(normalize=True)

False    0.955375
True     0.044625
Name: search_conducted, dtype: float64

In [23]:
pol.groupby('driver_gender')['search_conducted'].value_counts()

driver_gender  search_conducted
F              False               12625
               True                  247
M              False               32777
               True                 1531
Name: search_conducted, dtype: int64

In [24]:
pol.groupby('driver_gender')['search_conducted'].value_counts(normalize=True)

driver_gender  search_conducted
F              False               0.980811
               True                0.019189
M              False               0.955375
               True                0.044625
Name: search_conducted, dtype: float64

***

## Adding a second factor to the analysis

Even though the search rate for males is much higher than for females, it's possible that the difference is mostly due to a second factor.

For example, one might hypothesize that the search rate varies by violation type, and the difference in search rate between males and females is because they tend to commit different violations.

We can test this hypothesis by examining the search rate for each combination of gender and violation. If the hypothesis was true, we would find that males and females are searched at about the same rate for each violation. Let's find out below if that's the case!

In [25]:
pol.groupby('driver_gender')['violation'].value_counts(normalize=True)

driver_gender  violation          
F              Speeding               0.650637
               Moving violation       0.137819
               Equipment              0.112492
               Registration/plates    0.042107
               Other                  0.028900
               Seat belt              0.028045
M              Speeding               0.518800
               Moving violation       0.208377
               Equipment              0.136178
               Other                  0.059141
               Registration/plates    0.040544
               Seat belt              0.036959
Name: violation, dtype: float64

In [26]:
pol.groupby(['driver_gender','search_conducted'])['violation'].value_counts(normalize=True)

driver_gender  search_conducted  violation          
F              False             Speeding               0.657347
                                 Moving violation       0.135287
                                 Equipment              0.109941
                                 Registration/plates    0.040792
                                 Other                  0.028673
                                 Seat belt              0.027960
               True              Speeding               0.307692
                                 Moving violation       0.267206
                                 Equipment              0.242915
                                 Registration/plates    0.109312
                                 Other                  0.040486
                                 Seat belt              0.032389
M              False             Speeding               0.528572
                                 Moving violation       0.204625
                                 Equi

In [27]:
polf.groupby('search_conducted')['violation'].value_counts(normalize=True)

search_conducted  violation          
False             Speeding               0.657347
                  Moving violation       0.135287
                  Equipment              0.109941
                  Registration/plates    0.040792
                  Other                  0.028673
                  Seat belt              0.027960
True              Speeding               0.307692
                  Moving violation       0.267206
                  Equipment              0.242915
                  Registration/plates    0.109312
                  Other                  0.040486
                  Seat belt              0.032389
Name: violation, dtype: float64

In [28]:
polm.groupby('search_conducted')['violation'].value_counts(normalize=True)

search_conducted  violation          
False             Speeding               0.528572
                  Moving violation       0.204625
                  Equipment              0.132746
                  Other                  0.058913
                  Registration/plates    0.037862
                  Seat belt              0.037282
True              Speeding               0.309602
                  Moving violation       0.288700
                  Equipment              0.209667
                  Registration/plates    0.097975
                  Other                  0.064010
                  Seat belt              0.030046
Name: violation, dtype: float64

In [29]:
pol.groupby(['violation','driver_gender'])['search_conducted'].value_counts(normalize=True)

violation            driver_gender  search_conducted
Equipment            F              False               0.958564
                                    True                0.041436
                     M              False               0.931293
                                    True                0.068707
Moving violation     F              False               0.962796
                                    True                0.037204
                     M              False               0.938173
                                    True                0.061827
Other                F              False               0.973118
                                    True                0.026882
                     M              False               0.951700
                                    True                0.048300
Registration/plates  F              False               0.950185
                                    True                0.049815
                     M              F

***

## Counting protective frisks

During a vehicle search, the police officer may pat down the driver to check if they have a weapon. This is known as a ``"protective frisk."``

We'll first check to see how many times "Protective Frisk" was the only search type. Then, we'll use a string method to locate all instances in which the driver was frisked.

In [30]:
pol[pol['search_type'] == 'Protective Frisk']['search_type'].value_counts()

Protective Frisk    76
Name: search_type, dtype: int64

In [31]:
pol[pol['search_type'] != 'nan']['search_type'].head()

stop_datetime
2009-02-28 11:02:00    NaN
2012-05-17 13:45:00    NaN
2008-08-30 18:09:00    NaN
2009-04-13 22:36:00    NaN
2012-08-24 22:19:00    NaN
Name: search_type, dtype: object

In [32]:
pol['search_type'] = pol['search_type'].apply(str)
pol['search_type'].dtypes
#pol.info()

dtype('O')

In [33]:
pol['frisk'] = pol['search_type'].apply(lambda x: True if 'Protective Frisk' in x else False)
pol['frisk'].head()

stop_datetime
2009-02-28 11:02:00    False
2012-05-17 13:45:00    False
2008-08-30 18:09:00    False
2009-04-13 22:36:00    False
2012-08-24 22:19:00    False
Name: frisk, dtype: bool

In [34]:
sum(pol['frisk'] == True)

155

***

## Comparing frisk rates by gender

We'll compare the rates at which female and male drivers are frisked during a search. Are males frisked more often than females, perhaps because police officers consider them to be higher risk?

Before doing any calculations, it's important to filter the ``DataFrame`` to only include the relevant subset of data, namely stops in which a search was conducted.

In [35]:
searched = pol[pol['search_conducted'] == True]
searched.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1778 entries, 2007-10-28 01:01:00 to 2008-11-23 22:46:00
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1778 non-null   object 
 1   stop_date              1778 non-null   object 
 2   stop_time              1778 non-null   object 
 3   location_raw           1778 non-null   object 
 4   county_fips            0 non-null      float64
 5   fine_grained_location  0 non-null      float64
 6   police_department      1778 non-null   object 
 7   driver_gender          1778 non-null   object 
 8   driver_age_raw         1778 non-null   float64
 9   driver_age             1778 non-null   float64
 10  driver_race_raw        1778 non-null   object 
 11  driver_race            1778 non-null   object 
 12  violation_raw          1778 non-null   object 
 13  violation              1778 non-null   object 
 14  search_conducted    

In [36]:
searched['frisk'].mean()

0.08717660292463442

In [37]:
searched.groupby('driver_gender')['frisk'].mean()

driver_gender
F    0.068826
M    0.090137
Name: frisk, dtype: float64